You could just run the following notebook without hesitation

The following block are the functions to judge whether a graph has a bipartite

For each graph, we encode it as a string ```s = 'abcdefghi'```. If we have nodes labelled 0, 1, 2 on both sides, then ```s[3*i+j]``` is for the condition for node $i$ on the left and node $j$ on the right.

In [1]:
perms = [[0,4,8],[0,5,7],[1,5,6],[1,3,8],[2,4,6],[2,3,7]] #all 6 possible positions for bipartite matching.
def bipartite(s):
    # Given a fully known graph, return whether it has a perfect matching
    for indices in perms:
        if s[indices[0]]=="1" and s[indices[1]]=="1" and s[indices[2]]=="1":
            return True
    return False

def bipartite_with_unknown(s):
    # Return True/False if there is already a matching or there could never be a matching
    # Return None if there are both possibilities
    good_s=s.replace("x","1")
    bad_s=s.replace("x","0")
    if bipartite(bad_s): return True
    if bipartite(good_s): return None
    return False

In [2]:
import json
char="01x"
all_graph=set() # all possible graphs (encoding as strings)
for i in range (3**9):
    s=""
    n=i
    for j in range(9):
        s+=char[n%3]
        n=n//3
    all_graph.add(s)
classes=[]

char="01"
all_graph_cleared=set() # all possible cleared graph. The cleared is for "without unknown edges"
for i in range (2**9):
    s=""
    n=i
    for j in range(9):
        s+=char[n%2]
        n=n//2
    all_graph_cleared.add(s)
classes_cleared = []

In [3]:
perm=[[0,1,2],[0,2,1],[1,2,0],[1,0,2],[2,1,0],[2,0,1]]
# We do the randomized algorithm as in Virginia's paper:
# We do the permutations on nodes on the left and right, and both sides.
# Then, take the average

# Remember: the edge left i --- right j is represented by graph_string[3*i+j]

all_possible_perms = []
for perm_left in perm:
    for perm_right in perm:
        # generate permutation for edges: with left permutation perm_left and right permutation perm_right
        mapping={}
        for i in range(3):
            for j in range(3):
                # original left i --- right j
                # Now: left perm[i] --- right perm [j]
                mapping[i*3+j]=perm_left[i]*3+perm_right[j]
        all_possible_perms.append(mapping)

# Now, let's swap both sides
for perm_right in perm:
    for perm_left in perm:
        mapping={}
        for i in range(3):
            for j in range(3):
                # original left i --- right j
                # permutation: left perm[i] --- right perm [j]
                # switch both sides: right perm[j] --- left perm [i]
                mapping[i*3+j]=perm_right[j]*3+perm_left[i]
        all_possible_perms.append(mapping)

def equiv_class(s):
    # It returns a mapping for a graph string s. 
    # The keys are all the isomorphic graphs, 
    # and the value associated with each key is the mapping FROM (canonical) graph string s TO the key
    # That is, if kry:value, then key[value[i]] = s[i] 
    ans={}
    for perm in all_possible_perms: # perm is the permutation TO (canonical) graph string s FROM the key
        equiv = ""
        for ind in range(9):
            equiv += s[perm[ind]]
        perm2 = {v:k for k,v in perm.items()} #reverse permutation, FROM (canonical) graph string s TO the key
        ans[equiv] = perm2
    return ans

In [4]:
map_from_canonical={}
# Draw a graph as canonical from each class of isomirphic type.
# The dictionary above is canonical graph for each graph

# Categorize the graphs 
while len(all_graph)!=0:
    for graph in all_graph:
        break
    class_of_graph = equiv_class(graph)
    for equiv, mapping in class_of_graph.items():
        map_from_canonical[equiv] = mapping
    classes.append(list(class_of_graph.keys()))
    all_graph -= set(class_of_graph)

# Categorize the cleared graphs 
while len(all_graph_cleared)!=0:
    for graph in all_graph_cleared:
        break
    class_of_graph = equiv_class(graph)
    classes_cleared.append(list(class_of_graph.keys()))
    all_graph_cleared -= set(class_of_graph)

# So we only consider the canonical graphs, and perform same algorithm for isomorphic (equivalent) graphs
canonicals = {} # mapping a graph to the mapping to the canonical graph in the "types" set
types = set() # distinct types of graph
for graphs in classes:
    canonical = graphs[0]
    types.add(canonical)
    for graph in graphs:
        canonicals[graph] = canonical

In [5]:
print(f"We have originally {3**9} graphs, after branching, we only consider {len(types)} graphs")

We have originally 19683 graphs, after branching, we only consider 438 graphs


In [6]:
def guess(graph, algorithm):
    # Given an Algorithm written in the canonical graphs, And we have the query graph.
    # Algorithm is a type from "types" set to query index
    # Return the index to query next.
    # It is like: check the canonical --> algorithm query in canonical --> transform the index for the original graph
    canon = canonicals[graph]
    ask_in_canon = algorithm[canon]
    return map_from_canonical[graph][ask_in_canon]

def evaluation(algorithm):
    # For all the complete graphs, calculate the expected yes and no to query this.
    # Return the max expected number of yes and no for all graph, For all the graphs with and without matching.
    # Calculate separately if the graph itself is containing/not containing a matching.
    yes4yes = 0
    crit_yes4yes = None
    no4yes = 0
    crit_no4yes = None
    yes4no = 0
    crit_yes4no = None
    no4no = 0
    crit_no4no = None
    for graphs in classes_cleared:
        avg_num = len(graphs)
        yes = 0
        no = 0
        for graph in graphs:
            begin = "x"*9
            for _ in range(9):
                if bipartite_with_unknown(begin)!=None:
                    break
                guessing = guess(begin, algorithm)
                if graph[guessing] == "0": no+=1
                else: yes+=1
                begin = begin[:guessing]+graph[guessing]+begin[guessing+1:]
        avg_yes = yes/avg_num
        avg_no = no/avg_num
        if bipartite_with_unknown(graphs[0]) == True:
            if avg_yes > yes4yes:
                yes4yes = avg_yes
                crit_yes4yes = graphs[0]
            if avg_no > no4yes:
                no4yes = avg_no
                crit_no4yes = graphs[0]
        if bipartite_with_unknown(graphs[0]) == False:
            if avg_yes > yes4no:
                yes4no = avg_yes
                crit_yes4no = graphs[0]
            if avg_no > no4no:
                no4no = avg_no
                crit_no4no = graphs[0]
    return (yes4yes, crit_yes4yes, no4yes, crit_no4yes, yes4no, crit_yes4no, no4no, crit_no4no)

In [7]:
# de-duplicate: 
# That is, we have several graphs that has equivalent options
# We take only one from each equivalent options
# Just for an example, for an all-"x" graph, all edges are equivalent. We take one of them

valid = {} # mapping from graphs in "type" to indices for operation
# The structure valid type: 
# {canonical_graph_1 : 
#     {operation1: 
#         (canonical_graph_if_operation_1_is_no, canonical_graph_if_operation_1_is_yes)
#      operation2: ...},
#  canonical_graph_2: ...}
for graph in types:
    valid_op = {}
    reached = set()
    if bipartite_with_unknown(graph)==None: # That is, we need to query.
        ind = [i for i in range (9) if graph[i]=="x"]
        for i in ind:
            canonical_0 = canonicals[graph[:i]+"0"+graph[i+1:]] # graph with no
            canonical_1 = canonicals[graph[:i]+"1"+graph[i+1:]] # graph with yes
            if (canonical_0,canonical_1) not in reached:
                valid_op[i]=(canonical_0,canonical_1)
                reached.add((canonical_0,canonical_1))
        valid[graph]=valid_op
    else:
        valid[graph]={}

# branching the step with splitting T/F
# That is, for several graphs, if there is only one "x", we query that
# If there is more than one "x" and (for example: "00xx1x1xx"), if for some unqueried edge queried
# we can judge whether there is a matching, (for example, in index 2) then we choose that one-hit-ko edge.
valid2 = {}
okay = []
for graph in valid.keys():
    valid_op = valid[graph]
    flag = True
    if len(valid_op)>=2: # there are several edges
        for i in valid_op.keys():
            if bipartite_with_unknown(valid_op[i][0])!=None and bipartite_with_unknown(valid_op[i][1])!=None: # one-hit-ko
                if flag:
                    okay.append(graph)
                    flag = False
                    valid2[graph]={i : valid_op[i]}
    if flag:
        valid2[graph]=valid[graph]

In [8]:
'''
The following is a depreciated branching method. This thinks two step.
for graph in valid2.keys():
    valid_op = valid2[graph]
    True_OK=False
    True_OK_ind=None
    False_OK=False
    False_OK_ind=None
    True_UNK=False
    False_UNK=False
    if len(valid_op)>=2:
        for i in valid_op.keys():
            config0=bipartite_with_unknown(valid_op[i][0])
            config1=bipartite_with_unknown(valid_op[i][1])
            if config1==True:
                True_UNK = True
                if valid_op[i][0] in okay:
                    True_OK = True
                    True_OK_ind = i
            if config0==False:
                False_UNK=True
                if valid_op[i][1] in okay:
                    False_OK = True
                    False_OK_ind = i
        if True_OK==True and False_UNK==False:
            valid2[graph]={True_OK_ind:valid_op[True_OK_ind]}
        elif False_OK==True and not True_UNK==True:
            valid2[graph]={False_OK_ind:valid_op[False_OK_ind]}
'''

'\nThe following is a depreciated branching method. This thinks two step.\nfor graph in valid2.keys():\n    valid_op = valid2[graph]\n    True_OK=False\n    True_OK_ind=None\n    False_OK=False\n    False_OK_ind=None\n    True_UNK=False\n    False_UNK=False\n    if len(valid_op)>=2:\n        for i in valid_op.keys():\n            config0=bipartite_with_unknown(valid_op[i][0])\n            config1=bipartite_with_unknown(valid_op[i][1])\n            if config1==True:\n                True_UNK = True\n                if valid_op[i][0] in okay:\n                    True_OK = True\n                    True_OK_ind = i\n            if config0==False:\n                False_UNK=True\n                if valid_op[i][1] in okay:\n                    False_OK = True\n                    False_OK_ind = i\n        if True_OK==True and False_UNK==False:\n            valid2[graph]={True_OK_ind:valid_op[True_OK_ind]}\n        elif False_OK==True and not True_UNK==True:\n            valid2[graph]={False

In [9]:
rest = len([i for i in valid2.keys() if len(valid2[i])>1])
print(f"After branching, there are still {rest} out of {len(types)} graphs rest to consider.")

After branching, there are still 169 out of 438 graphs rest to consider.


In [10]:
# Here is a look of what valid looks like:
count = 5
for k,v in valid2.items():
    if count<0:
        break
    print(f"{k}:{v}")
    count-=1

x0x0x0000:{}
1x11x1000:{}
x0110xx1x:{}
x01x01000:{}
xx000111x:{0: ('10x100x10', '11x1x0001'), 8: ('1000x10x1', 'xx0111001')}
0x1xxxx0x:{1: ('010xxxxx0', 'xx00x1xx1'), 3: ('xx10x0x0x', '10xx1xxx0'), 4: ('x000xx1xx', 'x0xx1x1x0'), 5: ('10x0xxxx0', '0xxxx011x'), 6: ('x0100xxxx', 'xx1x10x0x'), 8: ('0xxxx00x1', '1x010xxxx')}


In [11]:
selections = valid2
number_of_known={i:set() for i in range(10)}
travelling_list = []
# order the types of graphs from less "x" to more "x"
for graph in types:
    number_of_known[graph.count("x")].add(graph)
for i in range(10):
    travelling_list+=number_of_known[i]

yes_vs_no={} # {graph:(number of yes, number of no)}
# Here suppose yes_vs_no[graph] = (a,b), and there are c "x"'s in the graph
# Then, we have a+b=2^c.
# If we choose every "x" arbitrarily, there are at most 2^c consequences.
# a out of 2^c consequences has a matching, and b of 2^c consequences are not.
for graph in travelling_list:
    if "x" not in graph:
        yes_vs_no[graph] = (1,0) if bipartite_with_unknown(graph) else (0,1)
    else:
        ind = graph.index("x")
        yes_graph = canonicals[graph[:ind]+"1"+graph[ind+1:]]
        no_graph = canonicals[graph[:ind]+"0"+graph[ind+1:]]
        total_yes = yes_vs_no[yes_graph][0]+yes_vs_no[no_graph][0]
        total_no = yes_vs_no[yes_graph][1]+yes_vs_no[no_graph][1]
        # That is why we order the graph from less "x" to more "x"... we can recurse!
        yes_vs_no[graph] = (total_yes, total_no)

pre_algorithm = {} # First rule out all the selections with smaller number of true graphs if the query edge is true.
for graph in valid2.keys():
    max_diff = -1
    max_indices = set()
    selection = valid2[graph]
    # This is a heuristic! A heuristic branching!
    # Choose a graph that has more splitting choice.
    # That is, the absolute value of the difference of "True" consequences between two outcome (0 and 1) for the chosen graph
    # is made biggest.
    for index in selection.keys():
        # Remember the structure of valid2?
        go_to = selection[index]
        yes_graph = go_to[1]
        no_graph = go_to[0]
        yes_for_yes = yes_vs_no[yes_graph][0]
        yes_for_no = yes_vs_no[no_graph][0]
        if abs(yes_for_yes-yes_for_no) > max_diff:
            max_indices = {index}
            max_diff = abs(yes_for_yes-yes_for_no)
        elif abs(yes_for_yes-yes_for_no) == max_diff:
            max_indices.add(index)
    pre_algorithm[graph] = {i:selection[i] for i in max_indices}

In [12]:
rest2 = len([i for i in valid2.keys() if len(pre_algorithm[i])>1])
print(f"After heuristic branching, there are still {rest2} out of {rest} graphs rest to consider. It is small enough to search!")

After heuristic branching, there are still 61 out of 169 graphs rest to consider. It is small enough to search!


In [13]:
# All the possible algorithms.
total_algo = {i:[] for i in types}

# Let's search! We store the algorithms starting with graph g in total_algo[g].
for graph in travelling_list:
    if pre_algorithm[graph] == {}:
        total_algo[graph] = set()
    else:
        option = pre_algorithm[graph]
        for i in option.keys():
            yes_to = option[i][0]
            no_to = option[i][1]
            yes_algo = total_algo[yes_to]
            no_algo = total_algo[no_to]
            if len(yes_algo)==0 and len(no_algo)==0:
                total_algo[graph].append({graph:i})
            elif len(yes_algo)==0 or len(no_algo)==0:
                for algo in list(yes_algo)+list(no_algo):
                    init={graph:i}
                    init.update(algo)
                    total_algo[graph].append(init)
            else:
                for y_algo in yes_algo:
                    for n_algo in no_algo:
                        init={graph:i}
                        init.update(y_algo)
                        init.update(n_algo)
                        total_algo[graph].append(init)
        #print("graph",graph,"has",len(algorithms[graph]),"possible algorithms")
#print("finish")

In [14]:
len(total_algo["xxxxxxxxx"])
# You can calculate yourself: there is a LOT of graphs (if I did not remembered wrong) 
# even for total_algo["0xxxxxxxx"] if there is bo heuristic branching!
# That is why I forfeited the last step of searching... you can do better!

512

In [15]:
'''
A deppreciated method to calculate one of the algorithm.
algorithm = {}
cur = {"x"*9}
for i in range(10):
    aux = set()
    for graph in cur:
        selection = selections[graph]
        if len(selection)>0:
            max_diff = -1
            max_index = -1
            for index in selection.keys():
                go_to = selection[index]
                yes_graph = go_to[1]
                no_graph = go_to[0]
                yes_for_yes = yes_vs_no[yes_graph][0]
                yes_for_no = yes_vs_no[no_graph][0]
                if abs(yes_for_yes-yes_for_no) > max_diff:
                    max_index = index
                    max_diff = abs(yes_for_yes-yes_for_no)
            algorithm[graph] = int(max_index)
            aux|=set(selection[max_index])
    cur = aux

evals = evaluation(algorithm)
yes4yes = evals[0]
no4yes = evals[2]
yes4no = evals[4]
no4no = evals[6]
import math
print((yes4yes + no4no)/2 + math.sqrt((yes4yes - no4no)**2/4 + no4yes*yes4no))
'''

'\nA deppreciated method to calculate one of the algorithm.\nalgorithm = {}\ncur = {"x"*9}\nfor i in range(10):\n    aux = set()\n    for graph in cur:\n        selection = selections[graph]\n        if len(selection)>0:\n            max_diff = -1\n            max_index = -1\n            for index in selection.keys():\n                go_to = selection[index]\n                yes_graph = go_to[1]\n                no_graph = go_to[0]\n                yes_for_yes = yes_vs_no[yes_graph][0]\n                yes_for_no = yes_vs_no[no_graph][0]\n                if abs(yes_for_yes-yes_for_no) > max_diff:\n                    max_index = index\n                    max_diff = abs(yes_for_yes-yes_for_no)\n            algorithm[graph] = int(max_index)\n            aux|=set(selection[max_index])\n    cur = aux\n\nevals = evaluation(algorithm)\nyes4yes = evals[0]\nno4yes = evals[2]\nyes4no = evals[4]\nno4no = evals[6]\nimport math\nprint((yes4yes + no4no)/2 + math.sqrt((yes4yes - no4no)**2/4 + no4y

In [16]:
import math
# Calculate the best algorithm and its eigenvalue.
# Search, Enumerate, and Evaluate!
min_eigen = 9
best_algo = None
for algo in total_algo["x"*9]:
    evals = evaluation(algo)
    yes4yes = evals[0]
    no4yes = evals[2]
    yes4no = evals[4]
    no4no = evals[6]
    eigen = (yes4yes + no4no)/2 + math.sqrt((yes4yes - no4no)**2/4 + no4yes*yes4no) # This is the largest eigenvalue.
    if eigen < min_eigen:
        best_algo = algo
        min_eigen = eigen

In [17]:
min_eigen # This is my bound!

5.9816453557597065

In [18]:
best_algo # This is my algorithm!

{'xxxxxxxxx': 0,
 'xxxxx0xxx': 2,
 'xxxxx0xx0': 2,
 '001xxxxxx': 3,
 '010xxxxx0': 6,
 'xxx0100x1': 0,
 'xx1xxx010': 3,
 '1x0x00xx1': 1,
 '101x1x00x': 8,
 'xxxxxx01x': 0,
 '0x0xx1xxx': 1,
 'xxx1xx01x': 2,
 '10xx1xxx0': 2,
 'xx1110x0x': 6,
 'x000111xx': 0,
 '101010x1x': 6,
 '0x1110001': 1,
 'xxxxxxxx1': 0,
 'xxx1xxx0x': 1,
 'x0xx0x1xx': 7,
 'xxx010x1x': 6,
 '10x100xxx': 2,
 '10x00x11x': 5,
 '11x10xx0x': 8,
 'x001x11x0': 0,
 '111001x0x': 6,
 'xxx01xxx1': 0,
 'x1x0xx0x1': 0,
 'xx01x1x10': 0,
 '100xx110x': 8,
 '0011xx101': 4,
 'xx1xxxx1x': 3,
 '1xxxx0x1x': 2,
 'xx111xx0x': 6,
 '01x01xxx1': 6,
 '1001xxx11': 4,
 '0101x1x10': 6}

In [19]:
evaluation(best_algo)

(3.6666666666666665,
 '111101001',
 1.6944444444444444,
 '101001110',
 2.5555555555555554,
 '001001110',
 4.111111111111111,
 '001001110')